In [1]:
import re
import pandas as pd
import json


In [9]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        for file in files:
            yield os.path.join(root, file)

# Example usage:
            
all_files = []
startpath = './full_texts/'
for filepath in list_files(startpath):
    if ".json" in filepath:
        all_files.append(filepath)
        

In [42]:

def find_digital_twin_definitions(text):
    regex_patterns = [

        # Regular expressions to extract the definitions
        r'\b(?:digital twin(?:s)?)\s+(?:is\s+)?(?:defined\s+as)\s+.+?(?=\.\s|\n|$)', # good one "digital twin is deined as"
        r'\b(?:defined)\s+(?:digital twin(?:s)?)\s+(?:as)\s+.+?(?=\.\s|\n|$)', # good one "xxx defined digital twin as"
        r'\b(?:digital twin(?:s)?)\s+(?:is\s+)?(?:defined\s+as)\s+.+?(?=\.\s|\n|$)', #good one "repeated"
        r'\bdigital twin(?:s)? is a .+?(?=\.\s|\n|$)', # good one "digital twin is a ....."
        # r'[^.!?]*(\bdigital twin(?:s)? is a .+?)(?=[.!?])',  # good but too slow
        r'\b(?:digital twin(?:s)?)\s+(?:is\s+)?(?:described\s+as)\s+.+?(?=\.\s|\n|$)', #good one "digital twin/s is/are described as ....."
        r'\b(?:digital twin(?:s)?)\s+(?:is\s+)?(?:characterized\s+by)\s+.+?(?=\.\s|\n|$)', #few results "digital twin is characterized by ...."
    ]

    definitions = []
    for pattern in regex_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)  # Ignore case and match newline characters
        definitions.extend(matches)


    return definitions


In [43]:
all_defs = []
iter = 0
all_definitions = []
for index, file in enumerate(all_files):
    try:
        title = d["full-text-retrieval-response"]["coredata"]["dc:title"]
    except:
        title = "unkonwn"
    data ={
        "file": file,
        "title": title
    }
    file_defs = []
    try:
        with open(file, 'r') as f:
            d = json.loads(f.read())
        definitions = find_digital_twin_definitions(d["full-text-retrieval-response"]["originalText"])
        if len(definitions) > 0 :
            for definition in definitions:
                if not definition in file_defs:
                    # print(iter, index,  definition)
                    file_defs.append(definition)
                    iter += 1
            data["defs"] = file_defs
            all_definitions.append(data)
            print(index,len(all_definitions),  end="\r")
    except Exception as e:
        pass

    with open("definitions.json", 'w') as fw:
        fw.write(json.dumps(all_definitions))
    # if len(all_definitions) > 30:
    #     break
        

In [29]:
all_definitions

[{'file': './full_texts/10.1053/j.semnuclmed.2022.10.007/10.1053_j.semnuclmed.2022.10.007.json',
  'metadata': {'eid': '1-s2.0-S0016508505006396',
   'dc:description': None,
   'openArchiveArticle': 'false',
   'prism:coverDate': '2005-04-30',
   'openaccessUserLicense': None,
   'prism:aggregationType': 'Journal',
   'prism:url': 'https://api.elsevier.com/content/article/pii/S0016508505006396',
   'link': [{'@_fa': 'true',
     '@rel': 'self',
     '@href': 'https://api.elsevier.com/content/article/pii/S0016508505006396'},
    {'@_fa': 'true',
     '@rel': 'scidir',
     '@href': 'https://www.sciencedirect.com/science/article/pii/S0016508505006396'}],
   'dc:format': 'application/json',
   'openaccessType': None,
   'pii': 'S0016-5085(05)00639-6',
   'prism:volume': '128',
   'prism:publisher': 'Published by Elsevier Inc.',
   'dc:title': 'Program pages for DDW 2005',
   'prism:copyright': 'Copyright © 2005 Published by Elsevier Inc. All rights reserved.',
   'openaccess': '0',
   'pr

"Dealing with climate change and its consequences on the environment have been one of the biggest challenges nowadays, where reducing the carbon footprint has been the focus of most sustainable strategies. The infrastructure is the dominant sector responsible for the total carbon footprint, accounting for approximately 70% of global carbon emissions. This study aims to illustrate the state-of-the-art of digital development and transformation of revealing and reducing carbon footprint in the Architecture, Engineering, Construction and Facility Management (AEC/FM) sectors. The digital tools for revealing and reducing infrastructure's carbon footprint would be summarized and also compared with other sectors, namely the tools for building and city. Current challenges and future development are also included."

In [ ]:
repeated = {}
unique = []
for i , definition in enumerate(all_definitions):
    if len(definition["defs"]) >0:
        print(len(definition["defs"]), definition["title"])
        for d in definition["defs"]:
            if not d.lower() in unique:
                unique.append(d)
            elif d.lower() in repeated.keys():
                repeated[d.lower()] += 1
            else:
                repeated[d.lower()] = 1
            print(d)
        print(" \n")
    

In [68]:
repeated

{'digital twin is a digital (virtual) copy of a physical (real) system or process created to optimize system performance': 1,
 'digital twin is a virtual model of a process, product or service': 1,
 'digital twin is a digital replica of a living or non-living physical entity': 3,
 'digital twin is a virtual replica that mimics the behaviour of an actual object or process throughout its existence': 1,
 'digital twin is a virtual instance of a physical system (twin) that is continually updated with the latter’s performance, maintenance, and health status data throughout the physical system’s life cycle': 1,
 'digital twin is a virtual representation of real-world entities and processes, synchronized at a specified frequency and fidelity': 1,
 'digital twin is a set of virtual information constructs that fully describes a potential or actual physical manufactured product from the micro atomic level to the macro geometrical level': 2,
 'digital twin is a systematic model with rank going fo

In [69]:
len(unique)

808